In [28]:
# Criar um navegador

# Importar/visualizar a base de dados

# Para cada produto na base de dados

    # Procurar esse produto no Google Shopping
        # Verificar se algum dos produtos do Google Shopping está dentro da faixa de preços
    # Procurar esse produto no Buscapé
        # Verificar se algum dos produtos do Buscapé está dentro da faixa de preços
        
# Salvar as ofertas boas em um dataframe (tabela)

# Exportar pro Excel

# Enviar por e-mail o resultado da tabela

### Passo 1: Importar Bibliotecas, Criar Navegador e Importar Base de Dados

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
from pathlib import Path
import win32com.client as win32
from datetime import datetime

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)
produtos_df = pd.read_excel("buscas.xlsx")

display(produtos_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,2500,4000
1,rtx 3060,zota galax,3500,5000


### Passo 2: Função de busca no Google

In [30]:
def google(nome_produto, termos_banidos, preco_minimo, preco_maximo):
    
    link = "https://www.google.com.br"
    
    # Listas
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_pesquisa = nome_produto.split(" ")
    lista_confirmacao_produto_invalido = []
    lista_pesquisa_valida = []
    
    # Janela inicial Google
    navegador.get(link)
    navegador.find_element(By.ID, 'APjFqb').send_keys(nome_produto)
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[4]/center/input[1]').click()
    time.sleep(1)
    
    # Janela de pesquisa 
    try:
        navegador.find_element(By.PARTIAL_LINK_TEXT, 'Shopping').click()
    except:
        print("Não há aba shopping")
    time.sleep(1)
    
    # Janela do Shopping
    
    # Lista de resultados para "Outros correspondentes"
    lista_resultados_outras_correspondencias = navegador.find_elements(By.CLASS_NAME, 'i0X6df')
    
    # Para cada resultado que aparece
    for resultado in lista_resultados_outras_correspondencias:
               
        nome_produto_web = resultado.find_element(By.CLASS_NAME, "tAxDx").text.lower()

        # Para cada termo banido na lista
        for termo_banido in lista_termos_banidos:

            # Se o termo banido não estiver no nome
            if termo_banido not in nome_produto_web:
                confirmacao_termo_banido = False
            else:
                confirmacao_termo_banido = True

            # Cria uma lista para confirmação do termo ser banido ou não
            lista_confirmacao_produto_invalido.append(confirmacao_termo_banido)

        # Para cada caracter no nome do produto
        for nome_produto in lista_termos_pesquisa:

            # Se o caracter não estiver no nome do produto da web
            if nome_produto not in nome_produto_web:
                confirmacao_pesquisa_invalida = True
            else:
                confirmacao_pesquisa_invalida = False

        # Adicona na lista para confirmação se tem todos os caracteres da pesquisa no nome do produto da web
        lista_confirmacao_produto_invalido.append(confirmacao_pesquisa_invalida)

        # Se a lista de verificação for diferente de verdadeiro, o produto passa por todos os reuisitos
        if True not in lista_confirmacao_produto_invalido:

            preco_produto_web = resultado.find_element(By.CLASS_NAME, "a8Pemb").text
            preco_produto_web = preco_produto_web.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco_produto_web = float(preco_produto_web)

            # Se o preço do produto estiver dentro do intervalo de mínimo e máximo
            if preco_minimo <= preco_produto_web <= preco_maximo:

                link_produto_web = resultado.find_elements(By.CLASS_NAME, "shntl")[1].get_attribute("href")
                
                # Adiciona o nome do produto, preço e o link da compra
                lista_pesquisa_valida.append((nome_produto_web, preco_produto_web, link_produto_web))

        lista_confirmacao_produto_invalido = []
        
    return lista_pesquisa_valida

### Passo 3: Função de busca no Buscapé

In [31]:
def buscape(nome_produto, termos_banidos, preco_minimo, preco_maximo):
    
    link = "https://www.buscape.com.br/"
    
    # Listas
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_pesquisa = nome_produto.split(" ")
    lista_confirmacao_produto_invalido = []
    lista_pesquisa_valida = []
    
    # Janela inicial do Buscapé
    navegador.get(link)
    navegador.find_element(By.CLASS_NAME, "AutoCompleteStyle_input__HG105").send_keys(nome_produto)
    navegador.find_element(By.CLASS_NAME, "AutoCompleteStyle_submitButton__GkxPO").click()
    time.sleep(2)
    
    # Tela de produtos
    lista_resultados_produtos = navegador.find_elements(By.CLASS_NAME, "SearchCard_ProductCard_Inner__7JhKb")
    
    # Para cada produto da lista de resultado
    for resultado in lista_resultados_produtos:
        
        nome_produto_web = resultado.find_element(By.CLASS_NAME, "SearchCard_ProductCard_Name__ZaO5o").text.lower()
        
        # Para cada termo banido na lista
        for termo_banido in lista_termos_banidos:

            # Se o termo banido não estiver no nome
            if termo_banido not in nome_produto_web:
                confirmacao_termo_banido = False
            else:
                confirmacao_termo_banido = True

            # Cria uma lista para confirmação do termo ser banido ou não
            lista_confirmacao_produto_invalido.append(confirmacao_termo_banido)

        # Para cada caracter no nome do produto
        for nome_produto in lista_termos_pesquisa:

            # Se o caracter não estiver no nome do produto da web
            if nome_produto not in nome_produto_web:
                confirmacao_pesquisa_invalida = True
            else:
                confirmacao_pesquisa_invalida = False

        # Adicona na lista para confirmação se tem todos os caracteres da pesquisa no nome do produto da web
        lista_confirmacao_produto_invalido.append(confirmacao_pesquisa_invalida)

        # Se a lista de verificação for diferente de verdadeiro, o produto passa por todos os reuisitos
        if True not in lista_confirmacao_produto_invalido:

            preco_produto_web = resultado.find_element(By.CLASS_NAME, "Text_MobileHeadingS__Zxam2").text
            preco_produto_web = preco_produto_web.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco_produto_web = float(preco_produto_web)

            # Se o preço do produto estiver dentro do intervalo de mínimo e máximo
            if preco_minimo <= preco_produto_web <= preco_maximo:

                link_produto_web = resultado.get_attribute("href")
                
                # Adiciona o nome do produto, preço e o link da compra
                lista_pesquisa_valida.append((nome_produto_web, preco_produto_web, link_produto_web))

        lista_confirmacao_produto_invalido = []
        
    return lista_pesquisa_valida

### Passo 4: Chamadas das funções

In [32]:
# Cria uma tabela vazia
tabela_produtos = pd.DataFrame()

# Para cada produto na base de dados
for produto in produtos_df.index:
    
    # Colunas da tabela 
    nome_produto = produtos_df.loc[produto, "Nome"]
    termos_banidos = produtos_df.loc[produto, "Termos banidos"]
    preco_minimo = produtos_df.loc[produto, "Preço mínimo"]
    preco_maximo = produtos_df.loc[produto, "Preço máximo"]
    
    # Chama a função do Google
    lista_pesquisa_google = google(nome_produto, termos_banidos, preco_minimo, preco_maximo)
    
    # Se retornar a lista de pesquisa
    if lista_pesquisa_google:
        
        # Adiciona os produtos na tabela
        tabela_google = pd.DataFrame(lista_pesquisa_google, columns=["Nome", "Preço", "Link"])
        tabela_produtos = pd.concat([tabela_produtos, tabela_google])
    
    # Chama a função do Buscapé
    lista_pesquisa_buscape = buscape(nome_produto, termos_banidos, preco_minimo, preco_maximo)
    
    # Se retornar a lista de pesquisa
    if lista_pesquisa_buscape:
        
        # Adiciona os produtos na tabela
        tabela_buscape = pd.DataFrame(lista_pesquisa_buscape, columns=["Nome", "Preço", "Link"])
        tabela_produtos = pd.concat([tabela_produtos, tabela_buscape])
    
display(tabela_produtos)

,Nome,Preço,Link
0,"vitrine: iphone 12 apple 64gb roxo tela 6,1 câ...",3369.00,https://www.google.com.br/url?url=https://www....
1,"vitrine: iphone 12 apple 64gb branco tela 6,1 ...",3369.00,https://www.google.com.br/url?url=https://www....
2,"apple iphone 12, 64gb - branco",3779.10,https://www.google.com.br/url?url=https://www....
3,iphone 12 64gb - azul - estou zerado,3059.10,https://www.google.com.br/url?url=https://www....
4,"vitrine: iphone 12 apple 64gb preto tela 6,1 c...",3369.00,https://www.google.com.br/url?url=https://www....
5,"iphone 12 64gb roxo tela 6,1 4g câmera traseir...",3298.99,https://www.google.com.br/url?url=https://www....
6,usado: iphone 12 64gb azul muito bom - trocafo...,3140.72,https://www.google.com.br/url?url=https://www....
7,usado: iphone 12 64gb preto excelente - trocafone,3709.00,https://www.google.com.br/url?url=https://www....
8,usado: iphone 12 64gb preto excelente - trocaf...,3167.12,https://www.google.com.br/url?url=https://www....
9,"iphone 12 64gb branco de vitrine tela 6,1&quot...",3298.99,https://www.google.com.br/url?url=https://www....


### Passo 5: Exportar Tabela Para o Excel

In [33]:
# Exportar tabela sem a coluna de índice
tabela_produtos.to_excel('Resultado Busca.xlsx', index = False)
print("Tabela exportada com sucesso!")

<ipython-input-33-e84d92341f2e>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  tabela_produtos.to_excel('Resultado Busca.xlsx', index = False)


Tabela exportada com sucesso!


### Passo 6: Enviar E-mail

In [34]:
# Verifica se existe produto na tabela
if len(tabela_produtos.index > 0):
    
    # Caminho do arquivo
    caminho_padrao = Path.cwd()
    caminho_arquivo = caminho_padrao / "Resultado Busca.xlsx"

    # Data
    data_hoje = date.today()
    data_hoje_formatada = data_hoje.strftime("%d/%m/%Y")
    
    # E-mail
    outlook = win32.Dispatch('outlook.application')
    email = outlook.CreateItem(0)
    email.to = "lsgalvao1000@gmail.com"
    email.Subject = f'Produto(s) Encontrado(s) na Faixa de Preço Desejada - {data_hoje_formatada}'
    email.HTMLBody = f"""
        <p>Prezados,</p>
        <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
        {tabela_produtos.to_html(index=False)}
        <p>Qualquer dúvida estou à disposição</p>
        <p>Att.,</p>
        """
    email.Attachments.Add(str(caminho_arquivo))
    
    email.Send()
    
    print("E-mail enviado com sucesso!")

# Fecha o navegador
navegador.quit()



E-mail enviado com sucesso!
